In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

# syllable_letters_translator = create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)

data = Data('s', accent_classification=True)
data.generate_data('syllabled_letters_accent_classification_train',
                   'syllabled_letters_accent_classification_test',
                   'syllabled_letters_accent_classification_validate', force_override=False)

LOADING DATA...
LOAD SUCCESSFUL!


In [8]:
%run ../../../prepare_data.py
gen = data.generator('train', 16)
test = next(gen)

In [21]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    syllable = ''
    for pos in range(0, len(el), 36):
        syllable += data.decode_x([el[pos:pos+36]], dictionary)
    print(syllable)
#     print(el)
    
#     print(syllable_dictionary[el.tolist().index(1)])

(16, 11, 252)
na
ja
ktu
a
lnej
ša







In [4]:
num_examples = 592886 # training set size

nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20



#conv_input_shape=(10, 252)
conv_input_shape=(10, 5168)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [ ]:
y_array_train = np.asarray(data.y_train)
accentuation_length_train = (y_array_train > 0).sum()
y_array_test = np.asarray(data.y_test)
accentuation_length_test = (y_array_test > 0).sum()
history = model.fit_generator(data.generator('train', batch_size), 
                              accentuation_length_train/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=accentuation_length_test/(batch_size * num_fake_epoch))

Epoch 1/400
1453/1452 [==============================] - 280s - loss: 0.2320 - actual_accuracy: 0.0483 - val_loss: 0.1656 - val_actual_accuracy: 0.2634
Epoch 2/400
1453/1452 [==============================] - 308s - loss: 0.1404 - actual_accuracy: 0.4408 - val_loss: 0.1221 - val_actual_accuracy: 0.5861
Epoch 3/400
1453/1452 [==============================] - 331s - loss: 0.1006 - actual_accuracy: 0.6335 - val_loss: 0.1025 - val_actual_accuracy: 0.6635
Epoch 4/400
1453/1452 [==============================] - 355s - loss: 0.0774 - actual_accuracy: 0.7340 - val_loss: 0.0833 - val_actual_accuracy: 0.7496
Epoch 5/400
1453/1452 [==============================] - 355s - loss: 0.0623 - actual_accuracy: 0.7935 - val_loss: 0.0790 - val_actual_accuracy: 0.7696
Epoch 6/400
1453/1452 [==============================] - 368s - loss: 0.0518 - actual_accuracy: 0.8280 - val_loss: 0.0729 - val_actual_accuracy: 0.7907
Epoch 7/400
1453/1452 [==============================] - 384s - loss: 0.0436 - actual_ac

1453/1452 [==============================] - 454s - loss: 0.0038 - actual_accuracy: 0.9886 - val_loss: 0.0592 - val_actual_accuracy: 0.9088
Epoch 55/400
1453/1452 [==============================] - 462s - loss: 0.0033 - actual_accuracy: 0.9903 - val_loss: 0.0572 - val_actual_accuracy: 0.9132
Epoch 56/400
1453/1452 [==============================] - 451s - loss: 0.0032 - actual_accuracy: 0.9917 - val_loss: 0.0660 - val_actual_accuracy: 0.9077
Epoch 57/400
1453/1452 [==============================] - 457s - loss: 0.0031 - actual_accuracy: 0.9915 - val_loss: 0.0600 - val_actual_accuracy: 0.9189
Epoch 58/400
1453/1452 [==============================] - 455s - loss: 0.0031 - actual_accuracy: 0.9909 - val_loss: 0.0578 - val_actual_accuracy: 0.9153
Epoch 59/400
1453/1452 [==============================] - 461s - loss: 0.0033 - actual_accuracy: 0.9904 - val_loss: 0.0579 - val_actual_accuracy: 0.9211
Epoch 60/400
1453/1452 [==============================] - 459s - loss: 0.0030 - actual_accuracy

1453/1452 [==============================] - 504s - loss: 0.0015 - actual_accuracy: 0.9959 - val_loss: 0.0547 - val_actual_accuracy: 0.9339
Epoch 108/400
1453/1452 [==============================] - 504s - loss: 0.0015 - actual_accuracy: 0.9958 - val_loss: 0.0659 - val_actual_accuracy: 0.9201
Epoch 109/400
1453/1452 [==============================] - 514s - loss: 0.0013 - actual_accuracy: 0.9966 - val_loss: 0.0673 - val_actual_accuracy: 0.9208
Epoch 110/400
1453/1452 [==============================] - 507s - loss: 0.0015 - actual_accuracy: 0.9962 - val_loss: 0.0670 - val_actual_accuracy: 0.9204
Epoch 111/400
1453/1452 [==============================] - 508s - loss: 0.0014 - actual_accuracy: 0.9966 - val_loss: 0.0746 - val_actual_accuracy: 0.9128
Epoch 112/400
1453/1452 [==============================] - 513s - loss: 0.0012 - actual_accuracy: 0.9968 - val_loss: 0.0696 - val_actual_accuracy: 0.9193
Epoch 113/400
1453/1452 [==============================] - 512s - loss: 0.0012 - actual_ac

1453/1452 [==============================] - 532s - loss: 0.0010 - actual_accuracy: 0.9970 - val_loss: 0.0735 - val_actual_accuracy: 0.9259
Epoch 160/400
1453/1452 [==============================] - 538s - loss: 8.4572e-04 - actual_accuracy: 0.9978 - val_loss: 0.0581 - val_actual_accuracy: 0.9368
Epoch 161/400
1453/1452 [==============================] - 537s - loss: 5.8757e-04 - actual_accuracy: 0.9985 - val_loss: 0.0784 - val_actual_accuracy: 0.9186
Epoch 162/400
1453/1452 [==============================] - 538s - loss: 8.7448e-04 - actual_accuracy: 0.9978 - val_loss: 0.0768 - val_actual_accuracy: 0.9208
Epoch 163/400
1453/1452 [==============================] - 534s - loss: 0.0012 - actual_accuracy: 0.9971 - val_loss: 0.0618 - val_actual_accuracy: 0.9291
Epoch 164/400
1453/1452 [==============================] - 534s - loss: 0.0011 - actual_accuracy: 0.9971 - val_loss: 0.0591 - val_actual_accuracy: 0.9331
Epoch 165/400
1453/1452 [==============================] - 528s - loss: 7.5906

In [6]:
name = '20_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [15]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)
# max_syllable = get_max_syllable(syllable_dictionary)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [8]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)